## Hist Gradient Boosting Hyperparameter Tuning (Recall)

In [45]:
# Load Libraries

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import recall_score

from sklearn.ensemble import HistGradientBoostingClassifier

import optuna

In [47]:
# Load Data

df = pd.read_csv('bank_4.csv', index_col=0)

In [49]:
# Train / Test Split

X = df.drop(columns=['churn', 'complain', 'umap_1', 'umap_2'])
y = df['churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [54]:
# Hyperparameter tuning

def objective(trial):
    
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 1.0),
        'max_iter': trial.suggest_int('max_iter', 100, 2000),
        'max_depth':trial.suggest_int('max_depth', 2, 50),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 32),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 10, 50),
        'l2_regularization': trial.suggest_float('l2_regularization', 0.0, 0.1)
    }

    model = HistGradientBoostingClassifier(
        **params,
        random_state=42)
    
    threshold = 0.31
    
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    scores = []
    
    for tr, te in skf.split(X_train, y_train):
        
        X_tr, X_te = X_train.iloc[tr], X_train.iloc[te]
        y_tr, y_te = y_train.iloc[tr], y_train.iloc[te]
        
        model.fit(X_tr, y_tr)
        prob = model.predict_proba(X_te)[:, 1]
        y_pred = np.where(prob < threshold, 0, 1)
        
        scores.append(recall_score(y_te, y_pred))
        
    return np.mean(scores)

In [56]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, show_progress_bar=True)

[I 2024-07-08 21:31:21,170] A new study created in memory with name: no-name-170febed-53d2-4d84-ab27-8a88e60004f8


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2024-07-08 21:31:24,408] Trial 0 finished with value: 0.6288343558282208 and parameters: {'learning_rate': 0.021644846573591232, 'max_iter': 169, 'max_depth': 43, 'min_samples_leaf': 32, 'max_leaf_nodes': 48, 'l2_regularization': 0.06720132148467962}. Best is trial 0 with value: 0.6288343558282208.
[I 2024-07-08 21:31:30,286] Trial 1 finished with value: 0.5196319018404908 and parameters: {'learning_rate': 0.6117201964045411, 'max_iter': 1082, 'max_depth': 16, 'min_samples_leaf': 26, 'max_leaf_nodes': 13, 'l2_regularization': 0.052490743456140354}. Best is trial 0 with value: 0.6288343558282208.
[I 2024-07-08 21:31:39,607] Trial 2 finished with value: 0.5067484662576687 and parameters: {'learning_rate': 0.8601640902989984, 'max_iter': 896, 'max_depth': 40, 'min_samples_leaf': 26, 'max_leaf_nodes': 26, 'l2_regularization': 0.09039413790573217}. Best is trial 0 with value: 0.6288343558282208.
[I 2024-07-08 21:32:11,636] Trial 3 finished with value: 0.5141104294478528 and parameters: {

In [58]:
print("Best trial:", study.best_trial)
print("Best hyperparameters:", study.best_params)

Best trial: FrozenTrial(number=55, state=1, values=[0.6306748466257668], datetime_start=datetime.datetime(2024, 7, 8, 21, 39, 37, 296716), datetime_complete=datetime.datetime(2024, 7, 8, 21, 39, 42, 618830), params={'learning_rate': 0.03134765616739056, 'max_iter': 377, 'max_depth': 40, 'min_samples_leaf': 29, 'max_leaf_nodes': 32, 'l2_regularization': 0.07646734966718845}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=1.0, log=False, low=0.001, step=None), 'max_iter': IntDistribution(high=2000, log=False, low=100, step=1), 'max_depth': IntDistribution(high=50, log=False, low=2, step=1), 'min_samples_leaf': IntDistribution(high=32, log=False, low=1, step=1), 'max_leaf_nodes': IntDistribution(high=50, log=False, low=10, step=1), 'l2_regularization': FloatDistribution(high=0.1, log=False, low=0.0, step=None)}, trial_id=55, value=None)
Best hyperparameters: {'learning_rate': 0.03134765616739056, 'max_iter': 377, 'max_depth':